In [ ]:
from WSN import *
import matplotlib.pyplot as plt
from tqdm import tqdm
from wsn_eval import *


In [ ]:
# plot_edges
def plot_edges(wsn : WSN):
    edges = []
    nodes = list(wsn.anchor_nodes)
    V = len(nodes)
    for ii, i in enumerate(nodes[:-1]):
        for j in nodes[ii+1:]:
            ri = wsn.nodes[i]
            rj = wsn.nodes[j]
            d = dist(ri, rj)
            edge = (d, i, j)
            heappush(edges, edge)
    
    for edge in edges:
        n1 = wsn.nodes[edge[1]]
        n2 = wsn.nodes[edge[2]]
        plt.plot([n1[0], n2[0]], [n1[1], n2[1]], c="orange")
    plt.scatter(wsn.nodes[:, 0], wsn.nodes[:, 1], c="blue")
    plt.xlim((0, 100))
    plt.ylim((0, 100))
    plt.show()


In [ ]:
# Initialize WSN
wsn = WSN(100, 7, std=0, D=142)
wsn.reset_nodes()
wsn.reset_anchors({1,2,3,4,5,6})
MST = wsn.find_MST()


In [ ]:
# modify_results
period = 10 / 100
def modify_results(results):
    new_results = [0] * len(results)
    for i, result in enumerate(results):
        # if result[1] >= 0:
            result0 = result[0]
            result1 = result[1] % period
            if result[1] < 0 or result1 < period / 2:
                result1 -= period
            result1 += np.random.normal(scale=0.5)
            new_results[i] = result0, result1
        # else:
        #     new_results[i] = result[0], result[1] % period - period
    return new_results


In [ ]:
# lots of localize functions
def localize_mTDOA_periodic(wsn: WSN):
    r0 = None
    for i in range(wsn.N):
        if i not in wsn.anchor_nodes:
            r0 = wsn.nodes[i]
            break
    if r0 is None:
        raise ValueError("wsn has no target node")
    results = wsn.transmit(0, r0)
    results = modify_results(results)
    H, b = wsn.get_mTDOA_H_and_b(results)
    est_pos = np.linalg.pinv(H) @ b
    return est_pos

def localize_mTDOA_periodic_with_tree(wsn: WSN):
    r0 = None
    for i in range(wsn.N):
        if i not in wsn.anchor_nodes:
            r0 = wsn.nodes[i]
            break
    if r0 is None:
        raise ValueError("wsn has no target node")
    
    MST = wsn.find_MST()
    results = wsn.transmit(0, r0)
    results = modify_results(results)
    H, b = wsn.get_mTDOA_H_and_b_with_tree(results)
    est_pos = np.linalg.pinv(H) @ b
    return est_pos

def localize_mTDOA(wsn: WSN):
    r0 = None
    for i in range(wsn.N):
        if i not in wsn.anchor_nodes:
            r0 = wsn.nodes[i]
            break
    if r0 is None:
        raise ValueError("wsn has no target node")
    results = wsn.transmit(0, r0)
    H, b = wsn.get_mTDOA_H_and_b(results)
    est_pos = np.linalg.pinv(H) @ b
    return est_pos


In [ ]:
# Localize with pairs
cluster_locations = np.array([
    [46, 15],
    [82, 45],
    [72, 84],
    # [36, 73],
    # [17, 48]
])
np.random.seed(1234)
num_clusters = len(cluster_locations)
cluster_size = 3
cluster_scale = 10
N = num_clusters * cluster_size + 1
nodes = np.empty((N, 2))
nodes[0] = [50, 50]
for c, cluster in enumerate(cluster_locations):
    nodes[c * cluster_size + 1:(c+1) * cluster_size + 1] = np.random.normal(loc=cluster, scale=cluster_scale, size=(cluster_size, 2))
clusters = np.array([nodes[c * cluster_size + 1: (c+1) * cluster_size + 1] for c in range(num_clusters)])
r0 = nodes[0]
# wsn.nodes = np.array([
#     [50, 50],
#     [46, 15],
#     [49, 17],
#     [82, 45],
#     [79, 49],
#     [72, 84],
#     [74, 87],
#     [36, 73],
#     [32, 70],
#     [17, 48],
#     [14, 48]
# ])
# N = 11
wsn.nodes = nodes
wsn.N = N
wsn.reset_anchors(set(range(1, N)))
def localize_by_clusters(wsn: WSN):
    H = np.zeros((num_clusters * (cluster_size-1), num_clusters + 3))
    b = np.zeros(num_clusters * (cluster_size-1))
    for c in range(num_clusters):             
        rc = clusters[c][0]
        for n in range(1, cluster_size):
                rn = clusters[c][n]
                # Generate time
                tn = (dist(rn, r0) - dist(rc, r0)) / wsn.c  
                Hn = np.zeros(num_clusters + 3)
                Hn[:2] = 2 * (rn - rc)
                Hn[2] = tn ** 2
                Hn[c + 3] = -2 * tn
                H[c * (cluster_size - 1) + n - 1] = Hn
                b[c * (cluster_size - 1) + n - 1] = rn @ rn - rc @ rc
    # return H, b
    est_pos = np.linalg.pinv(H) @ b
    return est_pos
est_pos = localize_by_clusters(wsn)
print(est_pos)
print(wsn.nodes[0])
# H, b = localize_by_clusters(wsn)


In [ ]:
def localize_mTDOA_quotient_estimator(wsn: WSN, period, epochs=5):
    r0 = None
    for i in range(wsn.N):
        if i not in wsn.anchor_nodes:
            r0 = wsn.nodes[i]
            break
    if r0 is None:
        raise ValueError("wsn has no target node")
    results = wsn.transmit(0, r0)
    mresults = modify_results(results)
    for e in range(epochs):
        H, b = wsn.get_mTDOA_H_and_b(mresults)
        est_pos = np.linalg.pinv(H) @ b
        speed = np.sqrt(est_pos[4])
        new_results = [0] * len(mresults)
        for i, result in enumerate(mresults):
            result0, result1 = result
            est_quot = dist(result0, est_pos[:2]) / speed
            



In [ ]:
# Initialize WSN
N = 7
wsn = WSN(100, N, verbose=False)
wsn.std = 0
wsn.D = 142


In [ ]:
# Massive test across Nrange
Nrange = range(6, 20)
iterations = 1000
num_err_types = 5
N_errs = np.zeros((len(Nrange), num_err_types))
for i, N in enumerate(Nrange):
    avg_loc_err = 0
    avg_center_err = 0
    avg_est_center_err = 0
    avg_time_err = 0
    avg_speed_err = 0
    avg_errs = np.zeros(num_err_types)
    for it in tqdm(range(iterations)):
        wsn.N = N
        wsn.reset_nodes()
        wsn.reset_anchors(set(range(1, N)))
        # est_pos = localize_mTDOA_periodic(wsn).flatten()
        # est_pos = localize_mTDOA_periodic(wsn).flatten()
        est_pos = wsn.localize_continuous(method="mTDOA", signal_type="fn").flatten()
        est_pos = np.concatenate((est_pos, np.zeros(2)))
        est_pos[3] = max(est_pos[3], 0)
        r0 = wsn.nodes[0]
        x = np.array([*r0, wsn.t0, wsn.c ** 2])
        avg = np.mean(wsn.nodes[1:], axis=0)
        loc_err = dist(est_pos[:2], x[:2])
        center_err = dist(avg, x[:2])
        est_center_err = dist(est_pos[:2], avg)
        time_err = abs(est_pos[2] - x[2])
        speed_err = abs(np.sqrt(est_pos[3]) - np.sqrt(x[3]))

        avg_errs += np.array([loc_err, center_err, est_center_err, time_err, speed_err])
    avg_errs /= iterations
    N_errs[i] = avg_errs




Results saved in N_errs.npy

In [ ]:
# Plotting
barWidth = 0.2
fig = plt.subplots(figsize=(10, 5))
colors = ("#eb4034", "#dea71b", "#b0f542", "#0d8c4a", "#0d338c")
labels = ("Position Error (mean Euclidean, m)",  "Center Error (mean Euclidean, m)", "Dist Betw. COM and Est Pos (mean euclidean, m)","Time Error (mean, s)", "Speed Error (mean, m/s)")

# for i in range(0, num_err_types):
b = 0
for i in [0,1,2]:
    br = list(Nrange)
    # br = [x - 1 for x in br]
    br = [x - 1 + barWidth * b for x in br]
    plt.bar(br[1:], N_errs[1:, i], color=colors[i], width=barWidth, label=labels[i])
    b += 1

plt.legend()
# plt.ylabel(labels[i])
plt.xlabel("N")
# plt.legend()
plt.show()

Test single `localize_mTDOA`

In [ ]:
# Initialize WSN and localize_mTDOA
N = 7
wsn.N = N
wsn.reset_nodes()
wsn.reset_anchors(set(range(1, N)))
est_pos = localize_mTDOA(wsn).flatten()


In [ ]:
# Plotting
plt.scatter(wsn.nodes[:, 0], wsn.nodes[:, 1], c=["blue" if i in wsn.anchor_nodes else "red" for i in range(wsn.N)], label="anchors")
plt.scatter(*est_pos[:2], c="orange", label="est_pos")
avg = np.mean(wsn.nodes[1:], axis=0)
plt.scatter(*avg, c="yellow", label="centerpoint")
plt.xlim((-20, 120))
plt.ylim((-20, 120))
plt.show()


Investigate why cTDOA isn't working. Clusters too close together?
With circular stimulus (single wave), it works as long as the entire waveform is considered.

In [1]:
from mutual_information import *
import mutual_information as mtin
from wsn_eval import *
from WSN import *
import multiprocessing as mp

In [2]:
mtin.solve_default_fn_equ()

In [ ]:
num_clusters = 4
cluster_size = 4
N = num_clusters * cluster_size
wsn = WSN(100, N, std=0, D=142, num_clusters=num_clusters, cluster_size=cluster_size, verbose=False)
np.random.seed(1234)
wsn.reset_clusters()
wsn.reset_anchors(set(range(N)))
root = np.array([55, 55])


Test cTDOA with single wave. <br>
Test details:
<ul>
    <li>500 iterations per </li>
    <li>num_clusters: 3-10 (multiprocessing) </li>
    <li>cluster_size: 3-4 </li>
    <li>cluster_spacing: 3-10 </li>
</ul>

Errors (num_errs=2):
<ul>
    <li>avg_distance_err (dist between est and true--[55, 55]--normalized by size of grid--[100]) </li>
    <li>avg_speed_err (dist between est speed and true speed--[1.6424885622140555]--normalized by true speed) </li>
</ul>

Each process will produce a numpy array of size `(len(cluster_size_range), len(cluster_spacing_range), num_err_types)`. Those arrays will be put into a larger array by the parent process of size `(len(num_clusters_range), len(cluster_size_range), len(cluster_spacing_range), num_err_types)`.

In [5]:
iterations = 500
num_clusters_range = range(3, 11)
cluster_size_range = range(3, 5)
cluster_spacing_range = range(3, 11)
num_err_types = 2

true_center = np.array([55, 55])
true_speed = 1.6424885622140555

errs = np.load("./cTDOA_errs.npy")

# with mp.Pool(len(num_clusters_range)) as pool:
#     results = pool.starmap(test_fn_clusters, (
#         (
#             num_clusters,
#             cluster_size_range,
#             cluster_spacing_range,
#             iterations,
#             true_center,
#             true_speed
#         ) for num_clusters in num_clusters_range)
#     )
#     errs = np.array(results)



Plot the errors. `shape=(len(num_clusters_range), len(cluster_size_range), len(cluster_spacing_range), num_err_types)`
<br>
For each type of error, one bar graph for cluster_size=3 and one for cluster_size=4. 

In [ ]:
# Plotting
barWidth = 0.1
err_labels = ("Position Error (mean Euclidean, m)", "Speed Error (mean, m/s)")
for err_type in range(num_err_types):

    for csize0, cluster_size in enumerate(cluster_size_range):
        b = 0
        fig = plt.subplots(figsize=(20, 5))
        colors = ("#eb4034", "#dea71b", "#b0f542", "#0d8c4a", "#0d338c", "#36f5ff", "#8a36ff", "#ff00e6", "#850080")
        for cspacing0, cluster_spacing in enumerate(cluster_spacing_range):
            br = list(num_clusters_range)
            br = [x + barWidth * b for x in br]
            plt.bar(br, errs[:, csize0, cspacing0, err_type], color=colors[csize0], width=barWidth, label=f"cluster_spacing={cluster_spacing}")
            b += 1

        plt.legend()
        plt.xlabel("num_clusters")
        plt.title(err_labels[err_type] + f"\nwith cluster_size={cluster_size}")
        plt.show()

In [4]:
errs

array([[[[27.09622133,  4.68513913]]]])

In [ ]:
iterations = 5
num_err_types = 4
avg_loc_err = 0
avg_center_err = 0
avg_est_center_err = 0
avg_errs = np.zeros(num_err_types)

for it in tqdm(range(iterations)):
    wsn.reset_clusters()
    est_pos = wsn.localize_continuous("cTDOA", signal_type="fn")
    loc_err = dist(est_pos[:2], root)
    center = np.mean(wsn.nodes, axis=0)
    center_err = dist(center, root)
    est_center_err = dist(est_pos[:2], center)

    avg_errs += np.array([loc_err, center_err, est_center_err])
avg_errs /= iterations
print(avg_errs)


In [ ]:
wsn.localize_continuous("cTDOA", signal_type="fn")